In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
      # print(os.path.join(dirname))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# creating path to training and testing files and listing number of images in the train and test files

path = "../input/planets-dataset/planet/planet/"
path_train = os.path.join(path, "train-jpg")
path_test = os.path.join(path, "test-jpg")
print('Amount of train files:', len(os.listdir(path_train)), '  Amount of test files:', len(os.listdir(path_test)))

In [3]:
# passing the train labels into a csv file

train_labels = pd.read_csv(path+'train_classes.csv', index_col= 0)
train_labels.head()

## Some checks on the label.

In [4]:
train_labels.isna().sum().plot(kind= 'bar')

In [5]:
train_labels.info()

# no missing values

In [6]:
# viewing some labels
list(train_labels.tags)[0:10]

# Imports

In [7]:
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [8]:
# visualizing some images
for images in os.listdir(path_train)[0:2]:
    img = cv2.imread(path+'train-jpg/{}'.format(images))
    plt.imshow(img)
    plt.show()
    
    

In [9]:
# function to flatten the inputs
flatten = lambda l: [item for sublist in l for item in sublist]

# getting unique classes from the tags column
classes = list(set(flatten([l.split(' ') for l in train_labels['tags'].values])))

labels = list(set(flatten([l.split(' ') for l in train_labels['tags'].values])))
label_numbs = {l: i for i, l in enumerate(labels)}

In [10]:
labels

In [11]:
x_train = []
y_train = []

In [12]:
for tr, tags in zip(os.listdir(path_train), list(train_labels.tags)):
    img = cv2.imread(path+'train-jpg/{}'.format(tr))
    
    targets = np.zeros(17)

    # some tags contain more than one feature so we put a one for each feature the tag contains.
    for t in tags.split(' '):
        targets[label_numbs[t]] = 1 
    x_train.append(cv2.resize(img, (32, 32)))
    y_train.append(targets)

In [13]:
y_train = np.array(y_train, np.uint8)

# scaling the data
x_train = np.array(x_train, np.float16) / 255.


In [14]:
# spliting data into train and test
no_in_train = 36000

In [15]:
x_train, x_validate, y_train, y_validate = x_train[:no_in_train], x_train[no_in_train:], y_train[:no_in_train], y_train[no_in_train:]

In [16]:
model = tf.keras.models.Sequential()

In [17]:
# adding layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.3))
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(64, activation= 'relu'))

model.add(Dense(17, activation='sigmoid'))

model.compile(loss='binary_crossentropy',  optimizer='adam', metrics=['accuracy'])
              
model.fit(x_train, y_train,
          batch_size=128,
          epochs=4,
          verbose=1,
          validation_data=(x_validate, y_validate))

In [20]:
# Predicting on the test data
x_test =[]
for test in os.listdir(path_test):
    img_test = cv2.imread(path+'test-jpg/{}'.format(test))
    
    x_test.append(cv2.resize(img, (32, 32)))

In [21]:
x_test[0]

In [22]:
x_test = np.array(x_test, np.float16) / 255.


In [24]:
test_predictions = model.predict(x_test, batch_size= 128)

In [27]:
test_predictions

In [26]:
test_df = pd.DataFrame({'image_name': [c for c in os.listdir(path_test)], 'tags':[test_prediction]})